<a href="https://colab.research.google.com/github/yuga2005/AI_Test/blob/main/Sentiment_analysis_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:

import torch
from transformers import pipeline, GPT2LMHeadModel, GPT2Tokenizer, BertForSequenceClassification, BertTokenizer
import nltk
from nltk.tokenize import sent_tokenize

# Ensure necessary NLTK data is downloaded
nltk.download('punkt')

# Load pre-trained sentiment analysis model and tokenizer
bert_model = BertForSequenceClassification.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")
bert_tokenizer = BertTokenizer.from_pretrained("nlptown/bert-base-multilingual-uncased-sentiment")

# Load GPT-2 model and tokenizer for generating replies
gpt2_tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
gpt2_model = GPT2LMHeadModel.from_pretrained("gpt2")

# Define function to read reviews from a text file
def read_reviews(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        reviews = file.readlines()
    return reviews

# Define function to perform sentiment analysis using BERT
def sentiment_analysis(text):
    inputs = bert_tokenizer(text, return_tensors="pt", truncation=True, padding=True)
    outputs = bert_model(**inputs)
    scores = outputs.logits.detach().numpy()[0]
    sentiment = scores.argmax()
    if sentiment in [0, 1]:  # Very Negative, Negative
        return "NEGATIVE"
    elif sentiment == 2:  # Neutral
        return "NEUTRAL"
    else:  # Positive, Very Positive
        return "POSITIVE"

# Define function to identify problems and generate replies
def analyze_reviews(reviews):
    problems = []
    replies = []

    for review in reviews:
        # Tokenize the review into sentences
        sentences = sent_tokenize(review)

        for sentence in sentences:
            # Analyze sentiment of each sentence
            sentiment = sentiment_analysis(sentence)

            # If the sentiment is negative or neutral, consider it a problem
            if sentiment in ['NEGATIVE', 'NEUTRAL']:
                problems.append(sentence)

                # Generate a reply based on the problem
                input_text = f"Customer complaint: {sentence}\nHotel response:"
                inputs = gpt2_tokenizer.encode(input_text, return_tensors="pt")
                outputs = gpt2_model.generate(inputs, max_length=100, num_return_sequences=1, no_repeat_ngram_size=2)
                reply = gpt2_tokenizer.decode(outputs[0], skip_special_tokens=True)
                replies.append(reply.replace(input_text, "").strip())

    return problems, replies

# Define function to print problems and replies
def print_problems_replies(problems, replies):
    for i, (problem, reply) in enumerate(zip(problems, replies)):
        print(f"Problem {i+1}: {problem}")
        print(f"Hotel Reply: {reply}")
        print()

# Main function to execute the analysis
def main():
    file_path = 'hotel_reviews.txt'  # Path to the file containing hotel reviews
    reviews = read_reviews(file_path)
    problems, replies = analyze_reviews(reviews)
    print_problems_replies(problems, replies)

if __name__ == "__main__":
    main()

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


FileNotFoundError: [Errno 2] No such file or directory: 'hotel_reviews.txt'